In [581]:
import pandas as pd
import numpy as np

stanza_df = pd.read_json('stanza-forms.json') 
columns = ['poem_title','tags']
stanza_tags_df = pd.read_csv('stanza_forms_poem_tags.txt', header=None, names=columns, sep="##")
verse_df = pd.read_json('verse-forms.json') 
verse_tags_df = pd.read_csv('verse_forms_poem_tags.txt', header=None, names=columns, sep="##")

/var/folders/ph/1rr0g3_55n966bkf9gyk1n9h0000gn/T/ipykernel_31620/2462150438.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  stanza_tags_df = pd.read_csv('stanza_forms_poem_tags.txt', header=None, names=columns, sep="##")
/var/folders/ph/1rr0g3_55n966bkf9gyk1n9h0000gn/T/ipykernel_31620/2462150438.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  verse_tags_df = pd.read_csv('verse_forms_poem_tags.txt', header=None, names=columns, sep="##")


In [582]:
verse_df = verse_df.sort_values(by='poem_title')
verse_tags_df = verse_tags_df.sort_values(by='poem_title')
stanza_df = stanza_df.sort_values(by='poem_title')
stanza_tags_df = stanza_tags_df.sort_values(by='poem_title')

In [583]:
zipped_tags = zip(verse_tags_df['poem_title'], verse_tags_df['tags'])
mapping = dict(zipped_tags)
verse_df['topics'] = verse_df['poem_title'].map(mapping)


zipped_tags = zip(stanza_tags_df['poem_title'], stanza_tags_df['tags'])
mapping = dict(zipped_tags)
stanza_df['topics'] = stanza_df['poem_title'].map(mapping)

In [584]:
collected_poems_df = pd.concat([stanza_df, verse_df], ignore_index=True)
collected_poems_df = collected_poems_df.dropna()

In [585]:
collected_poems_df['author'] = collected_poems_df['author'].str.title()

collected_poems_df['poem_text'] = collected_poems_df['poem_text'].apply('\n'.join)

collected_poems_df['topics'] = collected_poems_df['topics'].astype(str)
collected_poems_df['topics'] = collected_poems_df['topics'].apply(lambda x: x[1:-1].split(','))

# Explode the topics column
collected_poems_df_exploded = collected_poems_df.explode('topics')
collected_poems_df_exploded['topics'] = collected_poems_df_exploded['topics'].astype(str)
collected_poems_df_exploded['topics'] = collected_poems_df_exploded['topics'].apply(lambda x: x.strip(" '").lower())


In [586]:
# Get unique values
unique_items = collected_poems_df_exploded['topics'].unique()
print(unique_items)
forms = ['couplet', 'quatrain', 'free verse', 'ballad', 'landscapes & pastorals', 'tercet', 'villanelle', 'pastoral', 'epigram', 'dramatic monologue', 'pantoum', 'elegy', 'blank verse',  'epigraph', 'ode', 'ottava rima', 'sonnet', 'epistle', 'terza rima', 'ars poetica', 'aubade', 'prose poem', 'haiku', 'epic', 'sestina', 'ghazal', 'limerick', 'nursery rhymes']

collected_poems_df_exploded_filtered = collected_poems_df_exploded[collected_poems_df_exploded['topics'].isin(forms)]

zipped_tags = zip(collected_poems_df_exploded_filtered['poem_title'], collected_poems_df_exploded_filtered['topics'])
mapping = dict(zipped_tags)
collected_poems_df['form'] = collected_poems_df['poem_title'].map(mapping)

collected_poems_df = collected_poems_df.assign(collected_from="Poetry Foundation (Extracted)")
collected_poems_df.rename(columns={"url": "poem_link"}, inplace=True)


['social commentaries' 'gender & sexuality' 'relationships' 'lgbtq+'
 'arts & sciences' 'poetry & poets' 'stanza forms' 'rhymed stanza' 'humor'
 'war & conflict' 'meters' 'syllabic' 'mythology & folklore'
 'fairy-tales & legends' 'couplet' 'types/modes' 'nursery rhymes'
 'classics' 'timeless treasures' 'living' 'infancy' 'family & ancestors'
 'activities' 'eating & drinking' 'humor & satire' 'men & women'
 'travels & journeys' 'techniques' 'refrain' 'time & brevity'
 'disappointment' 'quatrain' 'series/sequence' 'music' 'anger' 'religion'
 'photography & film' 'free verse' 'love' 'desire & passion'
 'life choices' 'the mind' 'memory & nostalgia' 'nature' 'winter'
 'sports & outdoor activities' 'verse forms' 'ballad' 'optimism' 'death'
 'sorrow & grieving' 'blame' 'grief' 'heartache & loss'
 'infatuation & crushes' 'romantic love' 'ghosts & the supernatural'
 'horror' 'halloween' 'architecture & design' 'painting & sculpture'
 'ekphrasis' 'animals' 'philosophy' 'anxiety & insecurity' 'g

In [587]:
collected_poems_df = collected_poems_df.assign(topics_str=collected_poems_df['topics'])
collected_poems_df['topics_str'] = collected_poems_df['topics_str'].astype(str)
collected_poems_df.loc[collected_poems_df['form'] == 'landscapes & pastorals', 'form'] = 'pastoral'

strlist = ['free verse', 'dramatic monologue', 'blank verse', 'ottava rima', 'terza rima', 'ars poetica',  'prose poem', 'nursery rhymes',  'prose poem']
for item in strlist:
    collected_poems_df.loc[collected_poems_df['form'] == item, 'form'] = item.replace(" ", "-")
    
collected_poems_df.loc[(collected_poems_df['form'].isna()) & (collected_poems_df['topics_str'].str.contains('Common Measure')), 'form'] = 'common-measure'
collected_poems_df.loc[(collected_poems_df['form'].isna()) & (collected_poems_df['topics_str'].str.contains('Refrain')), 'form'] = 'refrain'
collected_poems_df.loc[(collected_poems_df['form'].isna()) & (collected_poems_df['topics_str'].str.contains('Mixed')), 'form'] = 'mixed'
collected_poems_df.loc[(collected_poems_df['form'].isna()) & (collected_poems_df['topics_str'].str.contains('Concrete or Pattern Poetry')), 'form'] = 'concrete-or-pattern-poetry'
collected_poems_df.loc[(collected_poems_df['form'].isna()) & (collected_poems_df['topics_str'].str.contains('Syllabic')), 'form'] = 'syllabic-verse'

In [588]:
collected_poems_df = collected_poems_df.reindex(columns=['author', 'poem_title', 'poem_text', 'form', 'topics', 'poem_link', 'collected_from'])

def get_extra_df():
    extra_characters = ['Translated from', 'Translated by']
    extra_df = pd.DataFrame()
    for item in extra_characters:     
        extra_df[item] = collected_poems_df.loc[:, 'poem_text'].str.contains(item)
    return collected_poems_df.loc[extra_df[extra_characters].any(axis=1)  == True]

extra_df = get_extra_df()
has_extra_lines = collected_poems_df[collected_poems_df['poem_title'].isin(extra_df['poem_title'])]
has_extra_lines.loc[:, 'poem_text'] = has_extra_lines['poem_text'].str.rsplit('\n', n=1).str[0]
zipped = zip(has_extra_lines['poem_title'], has_extra_lines['poem_text'])
mapping = dict(zipped)
collected_poems_df['updated_text'] = collected_poems_df['poem_title'].map(mapping)
collected_poems_df['poem_text'] = np.where(collected_poems_df['updated_text'].notna(), collected_poems_df['updated_text'], collected_poems_df['poem_text'])
collected_poems_df.drop('updated_text', axis=1, inplace=True)
collected_poems_df = collected_poems_df.dropna()

In [589]:
collected_poems_df

,author,poem_title,poem_text,form,topics,poem_link,collected_from
1,Rudyard Kipling,"""For All We Have And Are""","For all we have and are,\nFor all our children...",syllabic-verse,"['Social Commentaries', 'War & Conflict', 'S...",https://www.poetryfoundation.org/poems/57431/f...,Poetry Foundation (Extracted)
2,Mother Goose,"""Humpty Dumpty sat on a wall,""","Humpty Dumpty sat on a wall,\nHumpty Dumpty ha...",nursery-rhymes,"['Mythology & Folklore', 'Fairy-tales & Legen...",https://www.poetryfoundation.org/poems/46951/h...,Poetry Foundation (Extracted)
3,Mother Goose,"""Hush little baby, don't say a word,""","Hush little baby, don't say a word,\nPapa's go...",nursery-rhymes,"['Living', 'Infancy', 'Relationships', 'Fam...",https://www.poetryfoundation.org/poems/46950/h...,Poetry Foundation (Extracted)
4,,"""Hush-a-bye, baby, on the tree top,""","Hush-a-bye, baby, on the tree top,\nWhen the w...",nursery-rhymes,"['Living', 'Infancy', 'Stanza Forms', 'Coup...",https://www.poetryfoundation.org/poems/46977/h...,Poetry Foundation (Extracted)
6,Mother Goose,"""Pat-a-cake, pat-a-cake, baker's man,""","Pat-a-cake, pat-a-cake, baker's man,\nBake me ...",nursery-rhymes,"['Activities', 'Eating & Drinking', 'Stanza ...",https://www.poetryfoundation.org/poems/46962/p...,Poetry Foundation (Extracted)
...,...,...,...,...,...,...,...
3514,Edna St. Vincent Millay,“Time does not bring relief; you all have lied”,Time does not bring relief; you all have lied ...,sonnet,"['Love', 'Break-ups & Vexed Love', 'Desire &...",https://www.poetryfoundation.org/poems/46464/t...,Poetry Foundation (Extracted)
3515,Edna St. Vincent Millay,"“What lips my lips have kissed, and where, and...","What lips my lips have kissed, and where, and ...",sonnet,"['Love', 'Classic Love', 'Heartache & Loss',...",https://www.poetryfoundation.org/poems/46557/w...,Poetry Foundation (Extracted)
3516,Bertolt Brecht,"“When I’d reported to the couple, thus”","When I’d reported to the couple, thus\nThat up...",epigraph,"['Love', 'Break-ups & Vexed Love', 'Desire &...",https://www.poetryfoundation.org/poetrymagazin...,Poetry Foundation (Extracted)
3517,Jalal Al-Din Rumi,“Where did the handsome beloved go?”,"Where did the handsome beloved go?\nI wonder, ...",ghazal,"['Love', 'Heartache & Loss', 'Living', 'Mar...",https://www.poetryfoundation.org/poetrymagazin...,Poetry Foundation (Extracted)


In [590]:
duplicates = collected_poems_df[collected_poems_df.duplicated(subset=['poem_text'])]
duplicates

,author,poem_title,poem_text,form,topics,poem_link,collected_from
2620,Franklin Pierce Adams,A Ballad of Baseball Burdens,The burden of hard hitting. Slug away\n L...,ballad,"['Activities', 'Sports & Outdoor Activities',...",https://www.poetryfoundation.org/poems/45985/a...,Poetry Foundation (Extracted)
2621,Algernon Charles Swinburne,A Ballad of Death,"Kneel down, fair Love, and fill thyself with t...",ballad,"['Living', 'Death', 'Sorrow & Grieving', 'T...",https://www.poetryfoundation.org/poems/45277/a...,Poetry Foundation (Extracted)
2622,Algernon Charles Swinburne,"A Ballad of François Villon, Prince of All Bal...",Bird of the bitter bright grey golden morn\n ...,ballad,"['Love', 'Desire & Passion', 'Arts & Science...",https://www.poetryfoundation.org/poems/45278/a...,Poetry Foundation (Extracted)
2623,Thomas Moore,A Ballad: The Lake of the Dismal Swamp,"“They made her a grave, too cold and damp\nFor...",ballad,"['Love', 'Desire & Passion', 'Heartache & Lo...",https://www.poetryfoundation.org/poems/44780/a...,Poetry Foundation (Extracted)
2628,John Fuller,A Dialogue between Caliban and Ariel,Ar. Now you have been taught words and I am fr...,couplet,"['Arts & Sciences', 'Poetry & Poets', 'Theat...",https://www.poetryfoundation.org/poems/47600/a...,Poetry Foundation (Extracted)
...,...,...,...,...,...,...,...
3446,William Wordsworth,Yarrow Revisited,"The gallant Youth, who may have gained,\nOr se...",ballad,"['Nature', 'Seas, Rivers, & Streams', 'Liv...",https://www.poetryfoundation.org/poems/45566/y...,Poetry Foundation (Extracted)
3447,William Wordsworth,Yarrow Unvisited,\nFrom Stirling castle we had seen\nThe mazy F...,ballad,"['Nature', 'Landscapes & Pastorals', 'Seas, ...",https://www.poetryfoundation.org/poems/45568/y...,Poetry Foundation (Extracted)
3448,William Wordsworth,"Yarrow Visited. September, 1814",And is this—Yarrow?—This the stream\nOf which ...,ballad,"['Nature', 'Landscapes & Pastorals', 'Seas, ...",https://www.poetryfoundation.org/poems/45567/y...,Poetry Foundation (Extracted)
3478,Sir John Suckling,from A Ballad Upon A Wedding,"I tell thee, Dick, where I have been,\nWhere I...",ballad,"['Living', 'Marriage & Companionship', 'Rela...",https://www.poetryfoundation.org/poems/45243/a...,Poetry Foundation (Extracted)


In [591]:
total_poems = len(collected_poems_df) - len(duplicates)
print(f"Total poems: {total_poems}")

Total poems: 2387


In [ ]:
collected_poems_df = collected_poems_df.drop_duplicates(subset=['poem_text']).reset_index(drop=True) 
collected_poems_df.to_csv('../collected_poems.csv', sep='|', index=False)
print(len(collected_poems_df))

2387
